In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('iris-data.csv')

In [3]:
df.head()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 0 to 99
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sepal_length_cm  95 non-null     float64
 1   sepal_width_cm   95 non-null     float64
 2   petal_length_cm  95 non-null     float64
 3   petal_width_cm   95 non-null     float64
 4   class            95 non-null     object 
dtypes: float64(4), object(1)
memory usage: 4.5+ KB


In [5]:
df['class'].replace(["Iris-setosa","Iris-versicolor"], [1,0], inplace=True)

In [6]:
inp_df = df.drop(df.columns[[4]], axis=1)
out_df = df.drop(df.columns[[0,1,2,3]], axis=1)

#
scaler = StandardScaler()
inp_df = scaler.fit_transform(inp_df)

# Adding an extra column to be compatible with bias
new_c = (np.zeros(shape=(inp_df.shape[0],1)) + 1)
inp_df = np.concatenate((inp_df, new_c), axis=1)

#
X_train, X_test, y_train, y_test = train_test_split(inp_df, out_df, test_size=0.2, random_state=42)

In [9]:
X_train[:5]
y_train[:5]

,class
71,0
20,1
83,0
84,0
35,1


In [10]:
def model(X, W): # W is an array of weights
    # Logistic regression model implementation
    y = np.zeros(shape=(X.shape[0], 1))
    y = np.expand_dims(y, axis=1)
    y = 1/(1+ np.exp(-np.sum(W*X)))
    
    return y

In [11]:
def loss_bce(y_true, y_pred, W):
    # Binary cross entropy loss
    loss = -(1/y_train.shape[0])*np.sum(y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
    
    return loss

In [22]:
def create_mini_batches(X_train, y_train, batch_size):
    mini_batches = []
    data = np.hstack((X_train, y_train))
    np.random.shuffle(data)
    n_minibatches = data.shape[0] // batch_size
    i = 0
    for i in range(n_minibatches + 1):
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    if data.shape[0] % batch_size != 0:
        mini_batch = data[i * batch_size:data.shape[0]]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    return mini_batches

In [18]:
data = np.hstack((X_train, y_train))

In [25]:
def fit(num_weights, epoch=10, lr = 0.01):
    W = np.zeros(num_weights)
    for e in range(1, epoch+1):
        mini_batches = create_mini_batches(X_train, y_train, batch_size=16)
        for mini_batch in mini_batches:
            X_mini, y_mini = mini_batch
            y_pred = model(X_mini, W)
            print(e, ". Loss: ", loss_bce(y_train, y_pred, W), ", W: ", W)
            for i in range(W.shape[0]):
            # Calculate derivative against parameters [Implement yourself]
                x_i = X_mini[:,i]
                x_i= np.expand_dims(x_i,axis=1)
                dw_i = np.mean((y_pred-y_mini)*x_i)
            # Update parameters [Implement yourself]
                W[i] = W[i]-lr*dw_i
        
        # Plot the lines
        # Y_hat = w * X + b
        # plt.plot(X, Y_hat, label="Epoch: "+str(i))
    
    #plt.scatter(X, Y, label="dataset")
    #plt.legend()
    #plt.show()
    
    return W

In [26]:
W = fit(5)

1 . Loss:  class    0.693147
dtype: float64 , W:  [0. 0. 0. 0. 0.]
1 . Loss:  class    0.69283
dtype: float64 , W:  [-0.00045439  0.00328491 -0.00460411 -0.00470464  0.000625  ]
1 . Loss:  class    0.691524
dtype: float64 , W:  [-0.00213016  0.0061532  -0.00977804 -0.00950217  0.0006403 ]
1 . Loss:  class    0.696741
dtype: float64 , W:  [-0.00186471  0.01021571 -0.01424215 -0.01408502 -0.00052597]
1 . Loss:  class    0.687686
dtype: float64 , W:  [-0.00229478  0.01476128 -0.01914968 -0.0187497  -0.00067541]
1 . Loss:  class    0.689032
dtype: float64 , W:  [-0.00458512  0.01554237 -0.02346347 -0.02369986 -0.00258261]
2 . Loss:  class    0.687631
dtype: float64 , W:  [-0.00680614  0.01629196 -0.0276947  -0.02856435 -0.00429105]
2 . Loss:  class    0.687712
dtype: float64 , W:  [-0.00737438  0.0206128  -0.03262299 -0.03349785 -0.00443111]
2 . Loss:  class    0.701189
dtype: float64 , W:  [-0.00863714  0.02435777 -0.03750019 -0.03834125 -0.00460541]
2 . Loss:  class    0.695062
dtype: fl